In [ ]:
import csv
import pandas as pd
import numpy as np
import time

from tqdm import tqdm
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys

In [ ]:
data = []
with open('./data/musinsa_top_link_li.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        data.append(row)
data = data[0]
len(data)

In [ ]:
def get_review(driver, p_name, link, cnt):
    global on_page
    time.sleep(0.1)
    
    # 출력 용도
    try:
        on_page = driver.find_element(By.CSS_SELECTOR, '#reviewListFragment > div.nslist_bottom > div.pagination.textRight > div > a.paging-btn.btn.active').text
    except:
        # on_page = driver.find_element(By.CSS_SELECTOR, '#reviewListFragment > div > div.nslist_bottom > div.pagination.textRight > div > a.paging-btn.btn.active').text
        pass

    # 한 페이지의 전체 리뷰 리스트(최대 10개씩)
    req = driver.page_source
    soup = BeautifulSoup(req, 'html.parser')
    review10 = soup.find_all(class_='review-list')

    for i in range(len(review10)):
        if cnt==10:
                break
        
        try:
            # 사이즈 - 보통이에요
            if review10[i].find(class_='review-evaluation__list').find_all('li')[0].span.text == '보통이에요':
                size_rev = '보통'
            else:
                continue

            # 성별, 키, 몸무게, 사이즈
            body_info = review10[i].find(class_='review-profile__body_information').text.split(' · ')
            sex = body_info[0]
            height = body_info[1].replace('cm', '')
            weight = body_info[2].replace('kg', '')
            size = review10[i].find(class_='review-goods-information__option').text.strip() 

            # print(p_name, size, size_rev, sex, height, weight, link)

            review_list.append({
                '제품명':p_name,
                '사이즈':size,
                '리뷰':size_rev,
                '성별':sex,
                '키':height,
                '몸무게':weight,
                '링크':link
            })
            cnt += 1
        except:
            continue

    return on_page, cnt

In [ ]:
# 전체 코드

# 드라이버 옵션 생성 및 창 숨기기 옵션 추가
options = webdriver.ChromeOptions()
options.add_argument("headless")

review_list = []

# 0~1799(1800개)번째 리뷰 추출
for link in tqdm(data[486:1800]):
    try:
        # driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
        driver = webdriver.Chrome('../chromedriver_win32/chromedriver.exe', options=options)
        driver.get(link)
        
        # 추천 순 클릭
        driver.find_element(By.ID, 'reviewSelectSort').send_keys('추천 순')
        time.sleep(0.1)

        # 페이지 크롤링
        req = driver.page_source
        soup = BeautifulSoup(req, 'html.parser')

        # 제품명
        p_name = soup.find(class_='product_title').em.text.strip()

        # 리뷰 페이지 전체 수
        try:
            total_page = soup.find('div','box_page_msg').text.strip().split(' 페이지')[0]
            # print(total_page)
        except:
            driver.close()
            continue
        
        # 제품 당 10개 리뷰 가져오기 위한 카운트
        cnt = 0

        # 첫 페이지 크롤링
        on_page, cnt = get_review(driver, p_name, link, cnt)

        # 페이지 넘기기
        flag=True
        while flag:
            for i in range(4, 9):
                
                if (cnt== 10) | (on_page == total_page):
                    flag=False
                    break

                page_ = f'#reviewListFragment > div.nslist_bottom > div.pagination.textRight > div > a:nth-child({i})'
                
                try:
                    driver.find_element(By.CSS_SELECTOR, page_).send_keys(Keys.ENTER)
                except:
                    page_ = f'#reviewListFragment > div:nth-child(3) > div.nslist_bottom > div.pagination.textRight > div > a:nth-child({i})'
                    driver.find_element(By.CSS_SELECTOR, page_).send_keys(Keys.ENTER)
                    
                
                on_page, cnt = get_review(driver, p_name, link, cnt)

                

        driver.close()
    except:
        continue

In [ ]:
df = pd.DataFrame(review_list)
df

In [ ]:
df.to_csv('data/top_review_1_1800.csv', encoding='euc-kr')